# Planes intersection

## Making a plane

imports

In [216]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
#numpy for calculations
import numpy as np

In [217]:
"""utils.py"""
"""Create 2d mesh in sepecifies x and y axes limits, with number of points for every dimension separate."""
import numpy as np
def mesh2d(xlim, ylim, n=5):
    if isinstance(n, int):
        xx = np.linspace(xlim[0],xlim[1],n)
        yy = np.linspace(ylim[0],ylim[1],n)
    elif isinstance(n, list):
        xx = np.linspace(xlim[0],xlim[1],n[0])
        yy = np.linspace(ylim[0],ylim[1],n[1])
    else:
        raise Exception("Wrong number of points parameter")
    return np.meshgrid(xx, yy)

In [218]:
"""utils.py"""
"""Normalizes a 3d vector v, returns a 3d vector."""
def normalize(v):
    magnitude = np.sqrt(v[0]**2+v[1]**2+v[2]**2)
    if magnitude==0:
        raise ValueError("Zero vector cannot be normalized.")
    else:
        return v/magnitude

Lines are necessary as output of plane-plane intersection.

In [219]:
"""line.py"""
"""Line class for intersections simulation
@author Nick
@since 25.07.17"""
import plotly.graph_objs as go
import numpy as np
class Line:
    
    def __init__(self, vec, offset):
        self.vec = normalize(np.array(vec))
        self.offset = np.array(offset)
        t = np.linspace(0,1,2) #the parameter
        self.x = self.offset[0]+t*self.vec[0] #generate xpoints
        self.y = self.offset[1]+t*self.vec[1] #for y points
        self.z = self.offset[2]+t*self.vec[2] #for z points
    
    def getXYZ(self, layout=None):
        #TODO import data from layout
        t = np.linspace(0,1,2) #the parameter
        self.x = self.offset[0]+t*self.vec[0] #generate xpoints
        self.y = self.offset[1]+t*self.vec[1] #for y points
        self.z = self.offset[2]+t*self.vec[2] #for z points
        
    """Export the line into graphics object"""
    def goify(self,layout=None):
        line = go.Scatter3d(
            mode="lines",
            x=self.x,
            y=self.y,
            z=self.z,
            line = dict(
                color = ('rgb(205, 12, 24)'),
                width = 10)
            
        )
        return line

In [220]:
"""plane.py"""
"""Plane class for intersections simulation
@author Nick
@since 25.07.17"""
import plotly.graph_objs as go
#import .line #import local line module
import numpy as np

"""Planes are defined by their normal and offset vector."""
class Plane:
    
    def __init__(self, normal, offset):
        self.normal = normalize(np.array(normal))
        self.offset = np.array(offset)
    
    def getXYZ(self, xlim=[-1,1], ylim=[-1,1], zlim=[-1,1], n=10):
        if self.normal[2] == 0: #check if z is zero, then we have to generate x or y from other meshes
            if self.normal[1] == 0: #check if z and y is zero, then we have to generate x from yz mesh
                if self.normal[0] == 0: 
                    return ValueError("Normal vector is zero vector.")
                else:
                    #cannot generate z but can y, try generating y for xz mesh
                    y, z = mesh2d(ylim, zlim, n)
                    x = (np.dot(self.normal,self.offset)-self.normal[1]*y-self.normal[2]*z)/self.normal[0]
            else:
                #cannot generate z but can y, try generating y for xz mesh
                #self.normal[2] = 0.01 # TODO THIS IS VERY CRUDE
                x, z = mesh2d(xlim, zlim, n)
                y = ((np.dot(self.normal,self.offset)
                          - self.normal[0]*x
                          - self.normal[2]*z)
                              / self.normal[1])
        else:
            #try generating z
            x, y = mesh2d(xlim, ylim, n)
            #Generate plane z-values array
            z = (np.dot(self.normal,self.offset)-self.normal[0]*x-self.normal[1]*y)/self.normal[2]
        return x,y,z
    
    """Compute an intersection between this plane and 'other' plane. Returns Line object."""
    def intersection(self, other):
        print("Plane.intersection is depreciated. Please use algebra.intersection(plane1,plane2) instead.")
        if isinstance(other,Plane):
            #cross-product
            cross = np.cross(self.normal,other.normal)
            if np.all(cross==0):
                #planes are parallel or overlap
                raise ArithmeticError("Planes are parallel.")
            else:
                #sample point: x=0
                mat = [[self.normal[1],self.normal[2]],[other.normal[1], other.normal[2]]]
                axis = 0 #keep track of which axis we zero out
                #NOTE: this can raise np.linalg.LinAlgError exception when x=0 results in such a matrix
                #we have to check other cases
                if np.linalg.matrix_rank(mat) == 1:
                    mat = [[self.normal[0],self.normal[2]],[other.normal[0], other.normal[2]]]
                    axis = 1
                    if np.linalg.matrix_rank(mat) == 1:
                        mat = [[self.normal[0],self.normal[1]],[other.normal[0], other.normal[1]]]
                        axis = 2
                            
                rhs = [np.dot(self.normal,self.offset),np.dot(other.normal,other.offset)]
                sol = np.linalg.solve(mat,rhs)
                if axis == 0:
                    return Line(cross, [0,sol[0],sol[1]])
                if axis == 1:
                    return Line(cross, [sol[0],0,sol[1]])
                if axis == 2:
                    return Line(cross, [sol[0],sol[1],0])
        else:
            raise TypeError("'other' parameter is not a plane.")
    
    """Export the plane into graphics object"""
    def goify(self, layout=None):
        xx,yy,zz = self.getXYZ()
        surf = go.Surface(
            name="plane",
            x=xx,
            y=yy,
            z=zz
        )
        return surf

### Let's try it!

Equation of a plane: $$\vec{n} \cdot (\vec{r} - \vec{r_0}) = 0$$

In [221]:
plane1 = Plane([1.0,1.0,1.0],[0.5,0.0,0.0])
plane2 = Plane([1.0,-1.0,-1.0],[0.0,0.0,1.0])
intersection = plane1.intersection(plane2)
print(intersection.offset,intersection.vec)

Plane.intersection is depreciated. Please use algebra.intersection(plane1,plane2) instead.
[-0.25  0.    0.75] [ 0.          0.70710678 -0.70710678]


Generate traces

In [222]:
trace1 = plane1.goify()
trace2 = plane2.goify()
trace3 = plane1.intersection(plane2).goify()
data = [trace1,trace2,trace3]

Plane.intersection is depreciated. Please use algebra.intersection(plane1,plane2) instead.


Define layout

In [233]:
layout = dict(
    width=400,height=400,
    showlegend=False,
    font = dict(family="Verdana"),
    scene = dict(
        xaxis = dict(range=[-1, 1], autorange=False, zeroline=False),
        yaxis = dict(range=[-1, 1], autorange=False, zeroline=False),
        zaxis = dict(range=[-1, 1], autorange=False, zeroline=False),
        aspectmode = 'cube',
        camera = dict(center=dict(x=0,y=0,z=0),eye=dict(x=1,y=1,z=1))
    ),
    plot_bgcolor='rgb(255, 255, 255)'
)

Plot the figure

In [234]:
fig=go.Figure(data=data,layout=layout)
iplot(fig)

## Making intersections more general

We need points as well as planes and lines.

In [225]:
"""point.py"""
"""Point class to make returns from intersections more reasonable."""
class Point:
    def __init__(self,position):
        self.pos = np.array(position)
    
    def goify(self):
        pt = go.Scatter3d(
            mode="markers",
            x=np.array([self.pos[0]]),
            y=np.array([self.pos[1]]),
            z=np.array([self.pos[2]])
        )
        return pt

In [226]:
"""algebra.py"""
"""This contains algebra methods to calculate intersections etc."""
def intersection(obj1, obj2):
    """Intersection between planes"""
    if isinstance(obj1,Plane) and isinstance(obj2,Plane):
        return _plane_plane_intersection(obj1,obj2)
    elif isinstance(obj1,Line) and isinstance(obj2,Plane):
        return _plane_line_intersection(obj2,obj1)
    elif isinstance(obj1,Plane) and isinstance(obj2,Line):
        return _plane_line_intersection(obj1,obj2)
    else:
        raise TypeError("Invalid parameter types - please pass intersections.Point, intersections.Line or intersections.Plane.")

"""Private function; do not use.
plane-plane intersection submethod. Raises exceptions or returns Line."""
def _plane_plane_intersection(p1,p2):
    #cross-product
    cross = np.cross(p1.normal,p2.normal)
    if np.all(cross==0):
        #planes are parallel or overlap
        raise ArithmeticError("Planes are parallel.")
    else:
        #sample point: x=0
        mat = [[p1.normal[1],p1.normal[2]],[p2.normal[1], p2.normal[2]]]
        axis = 0 #keep track of which axis we zero out; 0=x, 1=y, 2=z
        #NOTE: linalg.solve can raise np.linalg.LinAlgError exception when x=0 results in singular matrix
        #we have to check some other cases
        #premise: the line has to intersect at least one of the planes: xy, yz or xz.
        if np.linalg.matrix_rank(mat) == 1:
            mat = [[p1.normal[0],p1.normal[2]],[p2.normal[0], p2.normal[2]]]
            axis = 1
            if np.linalg.matrix_rank(mat) == 1:
                mat = [[p1.normal[0],p1.normal[1]],[p1.normal[0], p2.normal[1]]]
                axis = 2  
        rhs = [np.dot(p1.normal,p1.offset),np.dot(p2.normal,p2.offset)]
        sol = np.linalg.solve(mat,rhs)
        if axis == 0:
            return Line(cross, [0,sol[0],sol[1]])
        if axis == 1:
            return Line(cross, [sol[0],0,sol[1]])
        if axis == 2:
            return Line(cross, [sol[0],sol[1],0])

"""Private function; do not use.
Intersection of a line an a plane. Raises exceptions or returns Point."""
def _plane_line_intersection(plane,line):
    check = np.dot(plane.normal,line.vec)
    if np.all(check==0):
        #plane and line are parallel or overlap
        raise ArithmeticError("Plane and line are parallel.")
    else:
        #there is an explicit formula for parameter of the line for which intersection is met:
        #t = (n . (rp-rv))/(n . v),
        #where n is normal to plane, v is line's direction, rp is plane offset, rv is vector offset
        t = np.dot(plane.normal,(plane.offset-line.offset))/np.dot(plane.normal,line.vec)
        return Point(line.offset+t*line.vec)

In [227]:
plane1 = Plane([1,1,1],[0,0,0])
plane2 = Plane([1,-1,1],[0,0,0])
intersec = intersection(plane1,plane2)
fig=go.Figure(data=[plane1.goify(),plane2.goify(),intersec.goify()],layout=layout)
iplot(fig)

In [228]:
plane = Plane([1,0.2,0.3],[0,0.5,0])
line = Line([1,1,1],[-.5,0,-0.5])
intersec = intersection(plane,line)
fig=go.Figure(data=[plane.goify(),line.goify(),intersec.goify()],layout=layout)
iplot(fig)

In [232]:
try:
    plane = Plane([1,0,0],[0,0.5,0])
    line = Line([0,1,1],[-.5,0,0.5])
    intersec = intersection(plane,line)
    fig=go.Figure(data=[plane.goify(),line.goify(),intersec.goify()],layout=layout)
    iplot(fig)
except:
    raise

ArithmeticError: Plane and line are parallel.

In [231]:
#testing layout dictionary access
print(layout["scene"]["yaxis"]["range"])

[-1, 1]
